In [2]:
import pandas as pd
import numpy as np

In [34]:
# 1. DADOS DE ENTRADA
# As datas futuras (previsões) são as mais importantes para o DCF.

dados = pd.read_csv('Itau IRFS.csv', sep=';', encoding='latin1')

dados.drop(columns=['31/12/2028'], inplace=True)
dados.drop(columns=['ITUB4'], inplace=True)

In [35]:
dados.head()

,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024,30/06/2024,30/09/2024,31/12/2024,31/03/2025,...,30/06/2026,30/09/2026,31/12/2026,31/03/2027,30/06/2027,30/09/2027,31/12/2027,31/03/2028,30/06/2028,30/09/2028
0,7.292,7.355,8.619,8.358,8.773,9.811,10.073,10.366,10.835,10.507,...,13.276.00,13.872.09,14.494.95,15.145.77,15.825.81,16.536.39,17.278.88,18.054.70,18.865.35,19.712.41
1,167.717,171.550,178.853,183.636,190.177,185.547,193.749,201.365,211.090,201.140,...,227.950.33,233.076.93,238.318.83,243.678.62,249.158.96,254.762.54,260.492.15,266.350.62,272.340.84,278.465.79
2,3.833,7.303,4.783,6.541,-4.630,8.202,7.616,9.725,-9.950,7.407,...,5.127,5.242,5.360,5.480,5.604,5.730,5.858,5.990,6.125,6.263
3,3.459,52.000,3.836,1.817,13.403,1.609,2.457,641.000,20.785,3.100,...,8.149,8.630,9.135,9.665,10.222,10.807,11.420,12.064,12.740,13.450


In [42]:
data = {
    'Data' : pd.to_datetime(dados.columns),
    'FCFE' : dados.iloc[3].values
}

/tmp/ipython-input-893233058.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  'Data' : pd.to_datetime(dados.columns),


In [43]:
data = pd.DataFrame(data)
data.set_index('Data', inplace=True)

print("Fluxos de Caixa Projetados (FCFE):")
print(data)
print("-" * 40)

Fluxos de Caixa Projetados (FCFE):
              FCFE
Data              
2022-12-31   3.459
2023-03-31    52.0
2023-06-30   3.836
2023-09-30   1.817
2023-12-31  13.403
2024-03-31   1.609
2024-06-30   2.457
2024-09-30   641.0
2024-12-31  20.785
2025-03-31     3.1
2025-06-30   6.447
2025-09-30   6.841
2025-12-31   7.256
2026-03-31   7.692
2026-06-30   8.149
2026-09-30   8.630
2026-12-31   9.135
2027-03-31   9.665
2027-06-30  10.222
2027-09-30  10.807
2027-12-31  11.420
2028-03-31  12.064
2028-06-30  12.740
2028-09-30  13.450
----------------------------------------


In [5]:
# 2. PREMISSAS DO VALUATION (ALTERE ESTAS VARIÁVEIS CONFORME SUA ANÁLISE)

# Custo de Equity (Ke) - Taxa de desconto anual, é a taxa de retorno que os acionistas esperam.
# Geralmente calculada via CAPM: Ke = Taxa Livre de Risco + Beta * (Prêmio de Risco de Mercado)
# Exemplo para o Brasil (valores ilustrativos):
taxa_livre_risco = 0.13  # Ex: Título público de longo prazo (NTN-B ou Selic Futura)
beta = 1.1               # Beta da empresa (sensibilidade ao mercado - QuantBrasil)
premio_risco_mercado = 0.0813 # Prêmio de risco para o mercado brasileiro (FGV)
custo_equity_anual = taxa_livre_risco + beta * (premio_risco_mercado) # Fórmula do CAPM

In [6]:
# Taxa de Crescimento na Perpetuidade (g)
# Taxa que se espera que o FCFE cresça para sempre. Deve ser conservadora.
# Geralmente um pouco abaixo ou igual ao crescimento nominal do PIB de longo prazo.
taxa_crescimento_perpetuo = 0.04

print("Premissas do Valuation:")
print(f"Custo de Equity Anual (Ke): {custo_equity_anual:.2%}")
print(f"Taxa de Crescimento na Perpetuidade (g): {taxa_crescimento_perpetuo:.2%}")
print("-" * 40)

Premissas do Valuation:
Custo de Equity Anual (Ke): 21.94%
Taxa de Crescimento na Perpetuidade (g): 4.00%
----------------------------------------


In [7]:
# 3. CÁLCULOS DO DCF

# Como seus dados são trimestrais, precisamos ajustar as taxas anuais para trimestrais.
custo_equity_trimestral = (1 + custo_equity_anual)**(1/4) - 1
taxa_crescimento_perpetuo_trimestral = (1 + taxa_crescimento_perpetuo)**(1/4) - 1

print("Taxas Ajustadas para o Período (Trimestral):")
print(f"Custo de Equity Trimestral: {custo_equity_trimestral:.4%}")
print(f"Crescimento na Perpetuidade Trimestral: {taxa_crescimento_perpetuo_trimestral:.4%}")
print("-" * 40)


Taxas Ajustadas para o Período (Trimestral):
Custo de Equity Trimestral: 5.0846%
Crescimento na Perpetuidade Trimestral: 0.9853%
----------------------------------------


In [8]:
# 3.1. Calcular o Valor Terminal
# Usamos o Modelo de Gordon Growth no último FCFE projetado.
ultimo_fcfe = df['FCFE'].iloc[-1]
ultimo_periodo = len(df)

valor_terminal = (ultimo_fcfe * (1 + taxa_crescimento_perpetuo_trimestral)) / (custo_equity_trimestral - taxa_crescimento_perpetuo_trimestral)

print("Cálculo do Valor Terminal:")
print(f"Último FCFE projetado: {ultimo_fcfe:,.2f}")
print(f"Valor Terminal no final do período de projeção: {valor_terminal:,.2f}")
print("-" * 40)


Cálculo do Valor Terminal:
Último FCFE projetado: 13,446.00
Valor Terminal no final do período de projeção: 331,239.69
----------------------------------------


In [9]:
# 3.2. Trazer os fluxos de caixa e o valor terminal a valor presente (VP)
df['Periodo'] = range(1, len(df) + 1)

# Calcula o VP de cada FCFE do período de projeção
df['VP_FCFE'] = df.apply(lambda row: row['FCFE'] / ((1 + custo_equity_trimestral)**row['Periodo']), axis=1)

# Calcula o VP do Valor Terminal
vp_valor_terminal = valor_terminal / ((1 + custo_equity_trimestral)**ultimo_periodo)


print("Cálculo do Valor Presente (VP):")
print(df[['Periodo', 'FCFE', 'VP_FCFE']].to_string(formatters={'FCFE':'{:,.2f}'.format, 'VP_FCFE':'{:,.2f}'.format}))
print(f"\nValor Presente do Valor Terminal: {vp_valor_terminal:,.2f}")
print("-" * 40)


Cálculo do Valor Presente (VP):
            Periodo      FCFE   VP_FCFE
Data                                   
2022-12-31        1  3,459.00  3,291.63
2023-03-31        2     52.00     47.09
2023-06-30        3  3,836.00  3,305.68
2023-09-30        4  1,817.00  1,490.04
2023-12-31        5 13,403.00 10,459.38
2024-03-31        6  1,609.00  1,194.87
2024-06-30        7  2,457.00  1,736.32
2024-09-30        8    641.00    431.07
2024-12-31        9 20,750.00 13,278.99
2025-03-31       10  3,100.00  1,887.86
2025-06-30       11  6,446.00  3,735.59
2025-09-30       12  6,840.00  3,772.12
2025-12-31       13  7,255.00  3,807.39
2026-03-31       14  7,690.00  3,840.41
2026-06-30       15  8,148.00  3,872.24
2026-09-30       16  8,629.00  3,902.41
2026-12-31       17  9,133.00  3,930.49
2027-03-31       18  9,663.00  3,957.36
2027-06-30       19 10,220.00  3,982.96
2027-09-30       20 10,804.00  4,006.82
2027-12-31       21 11,418.00  4,029.64
2028-03-31       22 12,061.00  4,050.61
2028-06-

In [12]:
# 3.3. Somar os valores presentes para obter o Equity Value
soma_vp_fcfe = df['VP_FCFE'].sum()
equity_value_total = (soma_vp_fcfe + vp_valor_terminal) * 1000000

In [13]:
# 4. RESULTADO FINAL
print("================ RESULTADO DO VALUATION ================")
print(f"Soma do VP dos FCFEs projetados: {soma_vp_fcfe:,.2f}")
print(f"VP do Valor Terminal: {vp_valor_terminal:,.2f}")
print(f"\nValor Intrínseco Total do Patrimônio Líquido (Equity Value): R$ {equity_value_total:,.2f}")
print("========================================================")

================ RESULTADO DO VALUATION ================
Soma do VP dos FCFEs projetados: 92,170.97
VP do Valor Terminal: 100,739.81

Valor Intrínseco Total do Patrimônio Líquido (Equity Value): R$ 192,910,782,727.41


In [14]:
# Calcular o Preço Justo por Ação
# Se você tiver o número de ações da empresa, pode calcular o valor por ação.
numero_de_acoes = 10784548883
preco_justo_por_acao = equity_value_total / numero_de_acoes
print(f"\nPreço Justo por Ação (Estimado): R$ {preco_justo_por_acao:,.2f}")


Preço Justo por Ação (Estimado): R$ 17.89


In [19]:
# Comparação do preço atual com o preço justo
# Baixando o preço atual do Yahoo Finance
import yfinance as yf
ticker = "ITUB4.SA"
mercado_ticker = "^BVSP"

preços = yf.download(ticker, period="1d")['Close']

/tmp/ipython-input-3484949794.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  preços = yf.download(ticker, period="1d")['Close']
[*********************100%***********************]  1 of 1 completed


In [20]:
print(f"Preço Atual de Mercado: R$ {preços.values[0].item():.2f}")

Preço Atual de Mercado: R$ 37.76


In [25]:
#Comparanado
PVP = preços.values[0].item() / preco_justo_por_acao


print(f"\nPreço Justo por Ação (Estimado): R$ {preco_justo_por_acao:,.2f}")
print(f"Preço Atual de Mercado: R$ {preços.values[0].item():.2f}")
print(f"P/VP: {PVP:.2f}")


Preço Justo por Ação (Estimado): R$ 17.89
Preço Atual de Mercado: R$ 37.76
P/VP: 2.11
